## Importing Dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import math

In [ ]:
# Assuming the file 'airbnb_Chicago.csv' is available in the current directory
# Note that this file is not same as original dataset (Some columns that seemed redundant are exluded)
# Reading the CSV file
df = pd.read_csv('airbnb_Chicago.csv')

In [ ]:
unique_rows = df.groupby(['Airbnb Host ID', 'Airbnb Property ID', 'Scraped Date']).size().reset_index(name='Counts')

# Get the number of unique rows
number_of_unique_rows = unique_rows.shape[0]
print(number_of_unique_rows)

120217


# Feature transformation -> Converting dates into ordinal categories

In [ ]:
import pandas as pd

# Sample DataFrame creation with 'Scraped Date' column
# Replace this with your actual DataFrame loading method
# df = pd.read_csv('path_to_your_csv_file.csv')

# Sample data for demonstration purposes

# Convert 'Scraped Date' to datetime
df['Scraped Date'] = pd.to_datetime(df['Scraped Date'], format='%d-%m-%Y')

# Define the start date of the first period
start_date = pd.Timestamp('2016-01-01')

# Function to calculate the period category
def calculate_period_category(date):
    # Calculate the number of quarters since the start date
    quarters_since_start = ((date.year - start_date.year) * 4) + ((date.month - 1) // 3)
    # Add 1 because we are starting with category 1
    return quarters_since_start + 1

# Apply the function to the 'Scraped Date' column to create a new 'Period Category' column
df['Period Category'] = df['Scraped Date'].apply(calculate_period_category)

# Display the DataFrame
df[['Scraped Date', 'Period Category']]

,Scraped Date,Period Category
0,2016-08-01,3
1,2016-11-01,4
2,2017-02-04,5
3,2017-05-01,6
4,2017-08-01,7
...,...,...
120212,2020-02-15,17
120213,2020-05-01,18
120214,2019-11-02,16
120215,2020-02-09,17


In [ ]:
df.drop('Scraped Date', axis=1, inplace=True)
df.head()

,Airbnb Host ID,Airbnb Property ID,superhost_period_all,scrapes_in_period,prev_scrapes_in_period,Superhost,superhost_change_lose_superhost,superhost_change_gain_superhost,rating_ave_pastYear,numReviews_pastYear,...,tractQuartilePrice_variance,booked_days_period_city,revenue_period_city,booked_days_period_tract,revenue_period_tract,tract_booking_share,tract_revenue_share,time_to_date_mean,prev_time_to_date_mean,Period Category
0,2140,10610,5,26,24,0,0,0,NaN,NaN,...,1.363636,104924,13169324,449,25390,0.004279,0.001928,19.739130,NaN,3
1,2140,10610,6,38,26,0,0,0,NaN,NaN,...,1.472727,68005,7337911,276,22274,0.004059,0.003035,84.833333,19.739130,4
2,2140,10610,7,23,38,0,0,0,NaN,NaN,...,1.472727,134202,17482964,340,19451,0.002533,0.001113,17.903226,84.833333,5
3,2140,10610,8,20,23,0,0,0,NaN,NaN,...,1.125000,132913,18431773,366,28349,0.002754,0.001538,47.000000,17.903226,6
4,2140,10610,9,20,20,0,0,0,NaN,NaN,...,1.472727,118284,15313425,395,27483,0.003339,0.001795,11.800000,47.000000,7



- Period 1: Q1 2016 (January, February, March 2016)
- Period 2: Q2 2016 (April, May, June 2016)
- Period 3: Q3 2016 (July, August, September 2016)
- Period 4: Q4 2016 (October, November, December 2016)
- Period 5: Q1 2017 (January, February, March 2017)
- Period 6: Q2 2017 (April, May, June 2017)
- Period 7: Q3 2017 (July, August, September 2017)
- Period 8: Q4 2017 (October, November, December 2017)
- Period 9: Q1 2018 (January, February, March 2018)
- Period 10: Q2 2018 (April, May, June 2018)
- Period 11: Q3 2018 (July, August, September 2018)
- Period 12: Q4 2018 (October, November, December 2018)
- Period 13: Q1 2019 (January, February, March 2019)
- Period 14: Q2 2019 (April, May, June 2019)
- Period 15: Q3 2019 (July, August, September 2019)
- Period 16: Q4 2019 (October, November, December 2019)
- Period 17: Q1 2020 (January, February, March 2020)
- Period 18: Q2 2020 (April, May, June 2020)

## Data Cleaning

# A. Imputing missing values

In [ ]:
# Assuming 'df' is your DataFrame
df = df.drop(['Created Date',], axis=1) #,'Airbnb Host ID','Airbnb Property ID'
# Convert 'Zipcode' column to object type
df['Zipcode'] = df['Zipcode'].astype(object)

# Columns to be removed
columns_to_remove = ['zip_hispanic_or_latino_anyrace', 'zip_white_nothispanic', 'zip_black_nothispanic', 'zip_asian_nothispanic']

# Remove the specified columns
df.drop(columns=columns_to_remove, axis=1, inplace=True)

# Display the DataFrame's information to verify the changes
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120217 entries, 0 to 120216
Data columns (total 90 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Airbnb Host ID                                    120217 non-null  int64  
 1   Airbnb Property ID                                120217 non-null  int64  
 2   superhost_period_all                              120217 non-null  int64  
 3   scrapes_in_period                                 120217 non-null  int64  
 4   prev_scrapes_in_period                            120217 non-null  int64  
 5   Superhost                                         120217 non-null  int64  
 6   superhost_change_lose_superhost                   120217 non-null  int64  
 7   superhost_change_gain_superhost                   120217 non-null  int64  
 8   rating_ave_pastYear                               100372 non-null  float64
 9   numR

In [ ]:
df.head()

,Airbnb Host ID,Airbnb Property ID,superhost_period_all,scrapes_in_period,prev_scrapes_in_period,Superhost,superhost_change_lose_superhost,superhost_change_gain_superhost,rating_ave_pastYear,numReviews_pastYear,...,tractQuartilePrice_variance,booked_days_period_city,revenue_period_city,booked_days_period_tract,revenue_period_tract,tract_booking_share,tract_revenue_share,time_to_date_mean,prev_time_to_date_mean,Period Category
0,2140,10610,5,26,24,0,0,0,NaN,NaN,...,1.363636,104924,13169324,449,25390,0.004279,0.001928,19.739130,NaN,3
1,2140,10610,6,38,26,0,0,0,NaN,NaN,...,1.472727,68005,7337911,276,22274,0.004059,0.003035,84.833333,19.739130,4
2,2140,10610,7,23,38,0,0,0,NaN,NaN,...,1.472727,134202,17482964,340,19451,0.002533,0.001113,17.903226,84.833333,5
3,2140,10610,8,20,23,0,0,0,NaN,NaN,...,1.125000,132913,18431773,366,28349,0.002754,0.001538,47.000000,17.903226,6
4,2140,10610,9,20,20,0,0,0,NaN,NaN,...,1.472727,118284,15313425,395,27483,0.003339,0.001795,11.800000,47.000000,7


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120217 entries, 0 to 120216
Data columns (total 90 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Airbnb Host ID                                    120217 non-null  int64  
 1   Airbnb Property ID                                120217 non-null  int64  
 2   superhost_period_all                              120217 non-null  int64  
 3   scrapes_in_period                                 120217 non-null  int64  
 4   prev_scrapes_in_period                            120217 non-null  int64  
 5   Superhost                                         120217 non-null  int64  
 6   superhost_change_lose_superhost                   120217 non-null  int64  
 7   superhost_change_gain_superhost                   120217 non-null  int64  
 8   rating_ave_pastYear                               100372 non-null  float64
 9   numR

In [ ]:
# Create imputers
mean_imputer = SimpleImputer(strategy='mean')  # for numerical columns
mode_imputer = SimpleImputer(strategy='most_frequent')  # for categorical/binary columns

# Iterate over each column in the DataFrame
for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        # Numerical columns: Use mean imputation (or median)
        df[col] = mean_imputer.fit_transform(df[[col]]).ravel()
    elif df[col].dtype == 'object' or df[col].dtype == 'bool':
        # For boolean columns, convert to integers, impute, then convert back
        if df[col].dtype == 'bool':
            df[col] = df[col].astype(int)
            df[col] = mode_imputer.fit_transform(df[[col]]).ravel()
            df[col] = df[col].astype(bool)
        else:
            # Categorical columns: Use mode imputation
            df[col] = mode_imputer.fit_transform(df[[col]]).ravel()


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120217 entries, 0 to 120216
Data columns (total 90 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Airbnb Host ID                                    120217 non-null  float64
 1   Airbnb Property ID                                120217 non-null  float64
 2   superhost_period_all                              120217 non-null  float64
 3   scrapes_in_period                                 120217 non-null  float64
 4   prev_scrapes_in_period                            120217 non-null  float64
 5   Superhost                                         120217 non-null  float64
 6   superhost_change_lose_superhost                   120217 non-null  float64
 7   superhost_change_gain_superhost                   120217 non-null  float64
 8   rating_ave_pastYear                               120217 non-null  float64
 9   numR

## B. Remove outliers

Finding specific methods to remove outliers for each row can be difficult. Have taken a simpler approach.

In [ ]:
df_backup = df

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120217 entries, 0 to 120216
Data columns (total 90 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Airbnb Host ID                                    120217 non-null  float64
 1   Airbnb Property ID                                120217 non-null  float64
 2   superhost_period_all                              120217 non-null  float64
 3   scrapes_in_period                                 120217 non-null  float64
 4   prev_scrapes_in_period                            120217 non-null  float64
 5   Superhost                                         120217 non-null  float64
 6   superhost_change_lose_superhost                   120217 non-null  float64
 7   superhost_change_gain_superhost                   120217 non-null  float64
 8   rating_ave_pastYear                               120217 non-null  float64
 9   numR

In [ ]:
# Applying outlier removal for all numerical columns, removing anything outside 3 standard deviations
for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        if(col == 'Superhost'):
            continue
        # Calculate the mean and standard deviation
        col_mean = df[col].mean()
        col_std = df[col].std()

        # Define the upper and lower bounds for outliers
        lower_bound = col_mean - 4 * col_std
        upper_bound = col_mean + 4 * col_std

        # Filter out outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66926 entries, 0 to 120127
Data columns (total 90 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Airbnb Host ID                                    66926 non-null  float64
 1   Airbnb Property ID                                66926 non-null  float64
 2   superhost_period_all                              66926 non-null  float64
 3   scrapes_in_period                                 66926 non-null  float64
 4   prev_scrapes_in_period                            66926 non-null  float64
 5   Superhost                                         66926 non-null  float64
 6   superhost_change_lose_superhost                   66926 non-null  float64
 7   superhost_change_gain_superhost                   66926 non-null  float64
 8   rating_ave_pastYear                               66926 non-null  float64
 9   numReviews_pastY

This script iterates over each column in your DataFrame and applies the IQR method to columns with numerical data (integers and floats). For boolean and categorical columns, it doesn't perform any outlier removal. You may need to adjust this script depending on the specific characteristics of your data.

The records are halved but the quality of the data is higher. (Can experiment with seeing what happens when we dont remove any at all)

# Categorical variables encoding

In [ ]:
# Assuming 'df' is your DataFrame and you want to drop the 'Created Date' column

# Drop the 'Created Date' column


# Display the DataFrame's information to verify the column has been dropped
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66926 entries, 0 to 120127
Data columns (total 90 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Airbnb Host ID                                    66926 non-null  float64
 1   Airbnb Property ID                                66926 non-null  float64
 2   superhost_period_all                              66926 non-null  float64
 3   scrapes_in_period                                 66926 non-null  float64
 4   prev_scrapes_in_period                            66926 non-null  float64
 5   Superhost                                         66926 non-null  float64
 6   superhost_change_lose_superhost                   66926 non-null  float64
 7   superhost_change_gain_superhost                   66926 non-null  float64
 8   rating_ave_pastYear                               66926 non-null  float64
 9   numReviews_pastY

In [ ]:
# Assuming 'df' is your DataFrame
# Select only the categorical columns for one-hot encoding
categorical_cols = df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to these columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Display the DataFrame's information to verify
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66926 entries, 0 to 120127
Columns: 270 entries, Airbnb Host ID to Neighborhood_Woodlawn
dtypes: bool(1), float64(86), uint8(183)
memory usage: 56.2 MB


<ipython-input-20-71dca5f1c295>:6: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


# Building models - Comaparing Linear and Backward linear regression

In [ ]:
df_encoded['Superhost'].value_counts()

0.0    40962
1.0    25964
Name: Superhost, dtype: int64

In [ ]:
df_encoded.head(30)

,Airbnb Host ID,Airbnb Property ID,superhost_period_all,scrapes_in_period,prev_scrapes_in_period,Superhost,superhost_change_lose_superhost,superhost_change_gain_superhost,rating_ave_pastYear,numReviews_pastYear,...,Neighborhood_South Shore,Neighborhood_Uptown,Neighborhood_Washington Heights,Neighborhood_Washington Park,Neighborhood_West Elsdon,Neighborhood_West Loop/Greektown,Neighborhood_West Ridge,Neighborhood_West Town/Noble Square,Neighborhood_Wicker Park,Neighborhood_Woodlawn
0,2140.0,10610.0,5.0,26.0,24.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
1,2140.0,10610.0,6.0,38.0,26.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
2,2140.0,10610.0,7.0,23.0,38.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
3,2140.0,10610.0,8.0,20.0,23.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
4,2140.0,10610.0,9.0,20.0,20.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
5,2140.0,10610.0,10.0,26.0,20.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
6,2140.0,10610.0,11.0,21.0,26.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
7,2140.0,10610.0,12.0,24.0,21.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
8,2140.0,10610.0,13.0,23.0,24.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0
9,2140.0,10610.0,14.0,18.0,23.0,0.0,0.0,0.0,4.746069,86.195141,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_encoded.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66926 entries, 0 to 120127
Data columns (total 270 columns):
 #    Column                                            Dtype  
---   ------                                            -----  
 0    Airbnb Host ID                                    float64
 1    Airbnb Property ID                                float64
 2    superhost_period_all                              float64
 3    scrapes_in_period                                 float64
 4    prev_scrapes_in_period                            float64
 5    Superhost                                         float64
 6    superhost_change_lose_superhost                   float64
 7    superhost_change_gain_superhost                   float64
 8    rating_ave_pastYear                               float64
 9    numReviews_pastYear                               float64
 10   numCancel_pastYear                                float64
 11   num_5_star_Rev_pastYear                           f

In [ ]:
df_encoded = df_encoded.drop(['Airbnb Host ID','Airbnb Property ID'], axis=1)

In [ ]:
df.head()

,Airbnb Host ID,Airbnb Property ID,superhost_period_all,scrapes_in_period,prev_scrapes_in_period,Superhost,superhost_change_lose_superhost,superhost_change_gain_superhost,rating_ave_pastYear,numReviews_pastYear,...,tractQuartilePrice_variance,booked_days_period_city,revenue_period_city,booked_days_period_tract,revenue_period_tract,tract_booking_share,tract_revenue_share,time_to_date_mean,prev_time_to_date_mean,Period Category
0,2140.0,10610.0,5.0,26.0,24.0,0.0,0.0,0.0,4.746069,86.195141,...,1.363636,104924.0,13169324.0,449.0,25390.0,0.004279,0.001928,19.739130,35.449269,3.0
1,2140.0,10610.0,6.0,38.0,26.0,0.0,0.0,0.0,4.746069,86.195141,...,1.472727,68005.0,7337911.0,276.0,22274.0,0.004059,0.003035,84.833333,19.739130,4.0
2,2140.0,10610.0,7.0,23.0,38.0,0.0,0.0,0.0,4.746069,86.195141,...,1.472727,134202.0,17482964.0,340.0,19451.0,0.002533,0.001113,17.903226,84.833333,5.0
3,2140.0,10610.0,8.0,20.0,23.0,0.0,0.0,0.0,4.746069,86.195141,...,1.125000,132913.0,18431773.0,366.0,28349.0,0.002754,0.001538,47.000000,17.903226,6.0
4,2140.0,10610.0,9.0,20.0,20.0,0.0,0.0,0.0,4.746069,86.195141,...,1.472727,118284.0,15313425.0,395.0,27483.0,0.003339,0.001795,11.800000,47.000000,7.0


In [ ]:
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66926 entries, 0 to 120127
Columns: 268 entries, superhost_period_all to Neighborhood_Woodlawn
dtypes: bool(1), float64(84), uint8(183)
memory usage: 55.1 MB


## A. Linear regression for predicting Rating

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df_encoded is your DataFrame
df = df_encoded

# Convert the boolean column 'Pets Allowed' to int (0 or 1)
df['Pets Allowed'] = df['Pets Allowed'].astype(int)

# Selecting features and target variable
X = df.drop(['Rating Overall'], axis=1)  # Features (all columns except 'Rating Overall')
y = df['Rating Overall']  # Target variable

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Adding a constant to the model for the intercept
X_train_sm = sm.add_constant(X_train)

# Creating and training the linear regression model using statsmodels
model_sm = sm.OLS(y_train, X_train_sm).fit()

# Printing the summary of the model
print(model_sm.summary())

# Predicting the target variable for the test set
X_test_sm = sm.add_constant(X_test)
y_pred = model_sm.predict(X_test_sm)

# Evaluating the model using sklearn metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nMean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


                            OLS Regression Results                            
Dep. Variable:         Rating Overall   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     359.3
Date:                Thu, 07 Dec 2023   Prob (F-statistic):               0.00
Time:                        21:20:06   Log-Likelihood:            -1.3221e+05
No. Observations:               53540   AIC:                         2.649e+05
Df Residuals:                   53284   BIC:                         2.672e+05
Df Model:                         255                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

## B. Backward Linear regression for predicting Rating

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df_encoded is your DataFrame
df = df_encoded

# Convert the boolean column 'Pets Allowed' to int (0 or 1)
df['Pets Allowed'] = df['Pets Allowed'].astype(int)

# Selecting features and target variable
X = df.drop(['Rating Overall'], axis=1)  # Features (all columns except 'Rating Overall')
y = df['Rating Overall']  # Target variable

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def backward_elimination(x, y, significance_level = 0.05):
    num_vars = len(x.columns)
    for i in range(0, num_vars):
        regressor_OLS = sm.OLS(y, sm.add_constant(x)).fit()
        max_p_value = max(regressor_OLS.pvalues)
        if max_p_value > significance_level:
            for j in range(0, num_vars - i):
                if (regressor_OLS.pvalues[j] == max_p_value):
                    x = x.drop(x.columns[j], axis=1)
                    break
    regressor_OLS.summary()
    return x

# Performing backward elimination
X_train_optimized = backward_elimination(X_train, y_train)

# Rebuild the model with the optimized set of features
X_train_sm = sm.add_constant(X_train_optimized)
model_sm = sm.OLS(y_train, X_train_sm).fit()
print(model_sm.summary())

# Predicting the target variable for the test set
# Note: We need to ensure that X_test contains the same features as X_train_optimized
X_test_sm = sm.add_constant(X_test[X_train_optimized.columns])
y_pred = model_sm.predict(X_test_sm)

# Evaluating the model using sklearn metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nMean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


                            OLS Regression Results                            
Dep. Variable:         Rating Overall   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     484.7
Date:                Thu, 07 Dec 2023   Prob (F-statistic):               0.00
Time:                        20:54:20   Log-Likelihood:            -1.3232e+05
No. Observations:               53540   AIC:                         2.650e+05
Df Residuals:                   53351   BIC:                         2.667e+05
Df Model:                         188                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------